# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


## Generate Cities List

In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [21]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

631

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [22]:
# Make the API call try to limit with rate limiter or Max function
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# Build query URL, rate limiter not working
query_url = f"{url}appid={weather_api_key}&units={units}&q="
# rate_limiter = RateLimiter(max_calls=50, period=60)


response = requests.get(query_url + "New York")
city_data = response.json()
city_data


{'coord': {'lon': -74.01, 'lat': 40.71},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 69.49,
  'feels_like': 65.21,
  'temp_min': 66.2,
  'temp_max': 71.6,
  'pressure': 1018,
  'humidity': 33},
 'visibility': 10000,
 'wind': {'speed': 3.36, 'deg': 0},
 'clouds': {'all': 1},
 'dt': 1600797601,
 'sys': {'type': 1,
  'id': 4610,
  'country': 'US',
  'sunrise': 1600771441,
  'sunset': 1600815191},
 'timezone': -14400,
 'id': 5128581,
 'name': 'New York',
 'cod': 200}

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [23]:
# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | lagos
Processing Record 3 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 4 of Set 1 | jatibonico
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | khatanga
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | graham
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | codrington
Processing Record 12 of Set 1 | yatou
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | peniche
Processing Record 15 of Set 1 | cairo
Processing Record 16 of Set 1 | lavrentiya
Processing Record 17 of Set 1 | bredasdorp
Processing Record 18 of Set 1 | narsaq
Processing Record 19 of Set 1 | saleaula
City not found. Skipping...
Processing Record 20 of Set 1 | norman wells
Processing Record 21 of Set 1 | tshikapa
Processing Record 22 of Set 1 | b

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [36]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.15,-70.92,48.20,61,40,23.04,CL,1600797598
1,lagos,6.58,3.75,78.80,94,75,5.82,NG,1600797692
2,jatibonico,21.94,-79.17,83.77,74,100,8.68,CU,1600797849
3,qaanaaq,77.48,-69.36,15.17,82,0,7.07,GL,1600797856
4,khatanga,71.97,102.50,46.08,74,18,9.62,RU,1600797862


In [40]:
#  Get the indices of cities that have humidity over 100%.

humidity_max = city_data_df["Humidity"].max()
humidity_max

100

In [34]:
#  Get the indices of cities that have humidity over 100%.
city_data_df[city_data_df.Humidity > 100]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [51]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

clean_city_data = city_data_df[city_data_df.Humidity < 100]
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.15,-70.92,48.20,61,40,23.04,CL,1600797598
1,lagos,6.58,3.75,78.80,94,75,5.82,NG,1600797692
2,jatibonico,21.94,-79.17,83.77,74,100,8.68,CU,1600797849
3,qaanaaq,77.48,-69.36,15.17,82,0,7.07,GL,1600797856
4,khatanga,71.97,102.50,46.08,74,18,9.62,RU,1600797862
...,...,...,...,...,...,...,...,...,...
569,dalianwan,39.03,121.69,62.60,88,99,2.24,CN,1600798116
570,gazanjyk,39.24,55.52,67.91,24,0,10.33,TM,1600798116
571,kieta,-6.22,155.63,79.90,73,83,3.09,PG,1600798116
572,saint anthony,45.02,-93.22,79.00,60,1,4.70,US,1600798026


In [53]:
# Max humidity columns greate than 100 have been removed

humidity_max = clean_city_data["Humidity"].max()
humidity_max

98

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression